In [3]:
import nibabel as nb
import numpy as np

In [2]:
#load a cifti image
input1='../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/tfMRI_MOVIE1_7T_AP/tfMRI_MOVIE1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
tr=1
img = nb.load(input1)
X = img.get_fdata()
X.shape

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(921, 170494)

In [4]:
#load a feature
input_f=np.load('../sourcedata/data/HCP_7T_movie_FIX/features/7T_MOVIE1_CC1_v2_mfs.npy')

In [13]:
input_f=np.load('../sourcedata/data/HCP_7T_movie_FIX/features/7T_MOVIE1_CC1_v2_pycochleagram_6_hrf.npy')

In [14]:
input_f.shape

(8, 1474)

In [ ]:

X = img.get_fdata() #load data from nii
X_t = X[17:] #trim beginning, first 17 TRs
Ybrain = X_t[:1009] #trim end to end of film    braintrain.append(s_brain[:-200,:]) #roughly 80 20 split, trim the last 200 TRs of each subject to save as test set
Ybrain = Ybrain[:,union_ind_ind[0]]
#     Ybrain_train = s_brain[:-200,:]
#     Ybrain_test = s_brain[-200:,:]
#as in https://nilearn.github.io/auto_examples/02_decoding/plot_miyawaki_encoding.html#sphx-glr-auto-examples-02-decoding-plot-miyawaki-encoding-py
Ybrain = np.nan_to_num(Ybrain)
estimator = RidgeCV(alphas=[0.1, 1.0, 10.0, 100])
cv = KFold(n_splits=5)
scores = []
for train, test in cv.split(X=X_as):
    train = train[2:-2] #remove the first and last 3 seconds of each test and train partition
    test = test[2:-2]
    #print('train, test')
    # we train the Ridge estimator on the training set
    # and predict the fMRI activity for the test set
    predictions = estimator.fit(X_as.reshape(-1, X_as.shape[1])[train], Ybrain[train]).predict(
        X_as.reshape(-1, X_as.shape[1])[test])
    # we compute how much variance our encoding model explains in each voxel
    scores.append(r2_score(Ybrain[test], predictions,
                           multioutput='raw_values'))
cut_score = np.mean(scores, axis=0)
#cut_score[cut_score < 0] = 0

cut_score_new = np.zeros((union_ind.shape[0]))
for i,ind in enumerate(union_ind_ind[0]):
    cut_score_new[ind]=cut_score[i]
all_scores.append(cut_score_new)
#make a temp valid cifti2img
#temp_img = nb.Cifti2Image(cut_score.reshape((1,91282)), img.header)
plot_surf(hcp.mesh.flat,\
    hcp.cortex_data(cut_score_new), cmap='viridis',symmetric_cmap=False, vmin=0, vmax=0.25, colorbar=True,avg_method='median',#figure=fig,\
    bg_map=hcp.mesh.sulc,title=f'voxel-wise Encoding Model, R2 {sub} 5-fold cv {featname}',\
    data_alpha=hcp.cortex_data(np.ones(cut_score_new.shape)),#np.ones(data_alpha05.shape)),\
    data_remove=hcp.cortex_data(data_remove),output_file=f'2021_02_05_low_level/{sub}-{featname}_ridgecv.png')
#plot_dscalar(temp_img, title=f'voxel-wise Encoding Model, R2 {sub} 5-fold cv {featname}',colorbar=True, vmin=0, vmax=1, plot_abs=False, threshold=None, cmap='bwr', output_file=f'2021_02_05_low_level/{sub}-{featname}_ridgecv.png')
all_scores=np.asarray(all_scores)
all_scores_mean=np.mean(all_scores,axis=0)
plot_surf(hcp.mesh.flat,\
hcp.cortex_data(all_scores_mean), cmap='viridis',symmetric_cmap=False, vmin=0, vmax=0.25, colorbar=True,avg_method='median',#figure=fig,\
bg_map=hcp.mesh.sulc,title=f'voxel-wise Encoding Model, R2 {sub} 5-fold cv {featname}',\
data_alpha=hcp.cortex_data(np.ones(cut_score_new.shape)),#np.ones(data_alpha05.shape)),\
data_remove=hcp.cortex_data(data_remove),output_file=f'2021_02_05_low_level/all-{featname}_ridgecv.png')

## 